In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from PIL import Image
import torch
from torchvision import transforms, models
from torch.utils.data import DataLoader, Dataset
import os

def get_img_paths(base_dir, folder_name):
    folder_path = os.path.join(base_dir, folder_name)
    img_paths = [os.path.join(folder_path, fname) for fname in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, fname))]
    return img_paths

base_dir = '/content/drive/My Drive'


data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


class CustomDataset(Dataset):
    def __init__(self, img_paths, class_to_idx, transform=None, test_mode=False):
        self.img_paths = img_paths
        self.class_to_idx = class_to_idx
        self.transform = transform
        self.test_mode = test_mode

    def __getitem__(self, index):
        img_path = self.img_paths[index]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)

        if self.test_mode:
            return image, img_path
        else:
            label = self.class_to_idx[img_path.split('/')[-2]]
            return image, label

    def __len__(self):
        return len(self.img_paths)

class_to_idx = {'hate_meme': 0, 'not_meme': 1, 'test_meme': 0, 'test_not_meme': 1}

train_dataset = CustomDataset(
    img_paths=get_img_paths(base_dir, 'hate_meme') + get_img_paths(base_dir, 'not_meme'),
    class_to_idx=class_to_idx,
    transform=data_transforms['train']
)

val_dataset = CustomDataset(
    img_paths=get_img_paths(base_dir, 'test_meme') + get_img_paths(base_dir, 'test_not_meme'),
    class_to_idx=class_to_idx,
    transform=data_transforms['val']
)


dataloaders = {
    'train': DataLoader(train_dataset, batch_size=4, shuffle=True),
    'val': DataLoader(val_dataset, batch_size=4, shuffle=False)
}


model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 2)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)


criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

num_epochs = 20
for epoch in range(num_epochs):
    print(f'Epoch {epoch+1}/{num_epochs}')
    print('-' * 10)

    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()
        else:
            model.eval()

        running_loss = 0.0
        running_corrects = 0


        for inputs, labels in dataloaders[phase]:
            inputs = inputs.to(device)
            labels = labels.to(device)


            optimizer.zero_grad()


            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)


                if phase == 'train':
                    loss.backward()
                    optimizer.step()


            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels)

        epoch_loss = running_loss / len(dataloaders[phase].dataset)
        epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

        print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

print('Training complete')


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/20
----------
train Loss: 0.7504 Acc: 0.6237
val Loss: 0.4175 Acc: 0.8333
Epoch 2/20
----------
train Loss: 0.5455 Acc: 0.7371
val Loss: 0.2573 Acc: 0.9167
Epoch 3/20
----------
train Loss: 0.5749 Acc: 0.7784
val Loss: 0.3796 Acc: 0.8182
Epoch 4/20
----------
train Loss: 0.6036 Acc: 0.7732
val Loss: 0.2929 Acc: 0.9091
Epoch 5/20
----------
train Loss: 0.4713 Acc: 0.7732
val Loss: 0.2995 Acc: 0.8939
Epoch 6/20
----------
train Loss: 0.3312 Acc: 0.8711
val Loss: 0.2572 Acc: 0.9167
Epoch 7/20
----------
train Loss: 0.3457 Acc: 0.8608
val Loss: 0.2498 Acc: 0.9167
Epoch 8/20
----------
train Loss: 0.5439 Acc: 0.8144
val Loss: 0.2791 Acc: 0.9167
Epoch 9/20
----------
train Loss: 0.3412 Acc: 0.8505
val Loss: 0.3363 Acc: 0.8864
Epoch 10/20
----------
train Loss: 0.6855 Acc: 0.7423
val Loss: 0.5016 Acc: 0.8712
Epoch 11/20
----------
train Loss: 0.6829 Acc: 0.7835
val Loss: 0.7567 Acc: 0.8030
Epoch 12/20
----------
train Loss: 0.4417 Acc: 0.8402
val Loss: 0.4188 Acc: 0.8561
Epoch 13/20
-

In [3]:

not_meme_val_dataset = CustomDataset(
    img_paths=get_img_paths(base_dir, 'test_not_meme'),
    class_to_idx=class_to_idx,
    transform=data_transforms['val']
)

hate_meme_val_dataset = CustomDataset(
    img_paths=get_img_paths(base_dir, 'test_meme'),
    class_to_idx=class_to_idx,
    transform=data_transforms['val']
)

specific_val_dataset = not_meme_val_dataset + hate_meme_val_dataset

specific_val_dataloader = DataLoader(specific_val_dataset, batch_size=4, shuffle=False)

predicted_labels_specific = []
true_labels_specific = []

for inputs, labels in specific_val_dataloader:
    inputs = inputs.to(device)
    labels = labels.to(device)

    with torch.no_grad():
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)


        predicted_labels_specific.extend(preds.cpu().numpy())
        true_labels_specific.extend(labels.cpu().numpy())


correct_predictions_specific = sum(p == t for p, t in zip(predicted_labels_specific, true_labels_specific))
total_predictions_specific = len(predicted_labels_specific)
accuracy_specific = correct_predictions_specific / total_predictions_specific

print(f'Accuracy on specific dataset: {accuracy_specific:.4f}')


Accuracy on specific dataset: 0.9394
